In [1]:
import sys
import ROOT as R
R.EnableImplicitMT()
%jsroot on

In [2]:
R.gSystem.Load("/data/HPS/lib/libMoller.dylib")
Moller = R.Moller()
print("Version of Moller Class:",Moller.GetVersion())

Version of Moller Class: 0.3.1


In [3]:
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/rafo_minidst/hps_007966.?_Moller_4.2_minidst.root"
#data_files_pattern = "/data/HPS/data/physrun2016/Moller/hps_007966.n_Moller_4.2.root"
#snapshot_file_name = "moller_rafo_filtered.root"
data_files_pattern = "/data/HPS/data/physrun2016/Moller/pass4kf/hps_007966.#_recon_5.2.1_minidst.root"
#snapshot_file_name = "moller_pass4kf_filtered.root"
ch_moller = R.TChain("MiniDST")
nf=0
for i in range(10):
    nf += ch_moller.Add(data_files_pattern.replace("#",str(i)))
# nf = ch_moller.Add(data_files_pattern)
n_events = ch_moller.GetEntries()
df_moller = R.RDataFrame(ch_moller)
df = df_moller
print(f"Data:  added {nf:3d} files to the chain, total of {n_events/1e6:7.2f}M events")

ch_mixed = R.TChain("MiniDST")
#nf_mixed = ch_mixed.Add("/data/HPS/data/physrun2016/Moller/moller_mixed_*.root")
nf_mixed = ch_mixed.Add("/data/HPS/data/physrun2016/Moller/moller_wide.root")
n_event_mix = ch_mixed.GetEntries()
df_mixed = R.RDataFrame(ch_mixed)
print(f"Mixed: added {nf_mixed:3d} files to the chain, total of {n_event_mix/1e6:7.2f}M events")

Data:  added  10 files to the chain, total of    4.72M events
Mixed: added   1 files to the chain, total of    4.00M events


# Moller Data Selection
We create three distrubutions from the data in dataframes. First is data with two electrons that have a track time that coincides within 2 ns, which makes it likely these are Moller pairs. For background we take two electrons that have a track time more than 10 ns apart, and separately a distribution with two electrons that come from different events.


In [4]:
#dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df),2.)
#df = df.Define("track_theta","RVec<double> out; for(int i=0;i<track_px.size();++i){out.push_back(atan2(sqrt(track_px[i]*track_px[i]+track_py[i]*track_py[i]),track_pz[i]));} return out;")
dfx = Moller.Select_El_Pairs(R.RDF.AsRNode(df), 0., 2.,"electron_pairs")\
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")
# Select delta time < 2ns.
dfxx = Moller.Select_El_Pairs(R.RDF.AsRNode(df),6., 100., "electron_pairs")\
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")  # Select delta time > 4ns and < 20ns.
dfm = Moller.Select_El_Pairs(R.RDF.AsRNode(df_mixed), 0., 100., "electron_pairs")\
    .Define("n_moller_pair","return electron_pairs.size();").Filter("n_moller_pair >= 1")

## Moller refinement cuts
For each of the three dataframes we now add a set of cuts. The cuts in the "R" series are likely okay to make. The cuts in the 
"X" series cut on the angles in ways that may affect the width of the Moller peak. The cuts are as follows:

* R1 - Cut on the electron momentum $|p_e| < 2 $ GeV
* R2 - Cut on the $\phi$ angle. If angle not specified, require the electrons to be in opposite volumes of HPS. With angle specified: $ |\phi_{e1} - \phi_{e2}| < \phi_{max}$.
* R3 - Cut on the calculated beam angle. $ (\vec{p}_{e1} + \vec{p}_{e2})_{\theta} < \theta_{max}$.
* R4 - Cut on the 3-momentum sum of the two electrons. $ 0.75 E_{b} < | \vec{p}_{e1} + \vec{p}_{e2} | < 1.15 E_{b}$.
* X1 - Brigid's cut on the sum of the theta angles, $ \theta_{min} < \theta_{e1} + \theta_{e2} < \theta_{max}$.
* X2 - Omar's fiducial cut on the tracks at the ECal, see Analysis note.

After each cut a standard set of variables is added to the dataframe in the Add_Four_Vectors method. This adds the 4-vectors for each electron rotated to account for the beam direction, the invariant mass of the 2 electrons etc. In some cuts above these variables are used, so we need to be careful to add them after a cut is made, and we need to make sure the ordering is correct.
 

In [5]:

# The cut for 2 and only 2 electrons is similar to the one done for the predefined Moller pairs, but only serves to cut the statistics. Uncomment below to get the
# statistics similar for the Beamspot constrained Moller pairs.
#dfx = dfx.Define("n_emin","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Filter("n_emin == 2").Filter("electron_pairs.size() == 1")
#dfxx = dfxx.Define("n_emin","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Filter("n_emin == 2").Filter("electron_pairs.size() == 1")
d = [dfx, dfxx, dfm]
for i in range(len(d)):
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "electron_pairs","part_","p4")
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 2.0, 2.306*0., 2.306*2., "electron_pairs", "el_pairs_r1")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r1","part_","r1_p4")
    d[i] = Moller.Refine_El_Pairs_2(R.RDF.AsRNode(d[i]), "el_pairs_r1", "el_pairs_r2")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r2","part_","r2_p4")
    d[i] = Moller.Refine_El_Pairs_3(R.RDF.AsRNode(d[i]), 0.01, "el_pairs_r2", "r2_", "el_pairs_r3")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r3","part_","r3_p4")
    d[i] = Moller.Refine_El_Pairs_1(R.RDF.AsRNode(d[i]), 2.0, 2.306*0.85, 2.306*1.1, "el_pairs_r3", "el_pairs_r4")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_r4","part_","r4_p4")
    
    # These cuts should not be done.
    d[i] = Moller.Refine_El_Pairs_X(R.RDF.AsRNode(d[i]), 0.040, 0.048, "r2_p4tht1", "r2_p4tht2" ,"el_pairs_r3", "el_pairs_rx1")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_rx1","part_","rx1_p4")
    d[i] = Moller.Refine_El_Pairs_X2(R.RDF.AsRNode(d[i]),"el_pairs_r3", "el_pairs_rx2")
    d[i] = Moller.Add_Four_Vectors(R.RDF.AsRNode(d[i]), -0.0302, "el_pairs_rx2","part_","rx2_p4")

dfx, dfxx, dfm = d

In [6]:
Particle_names=["Final state particle KF", "UC V0 vertex KF", "BSC V0 vertex KF", "TC V0 vertex KF", "UC Moller vertex KF",
"BSC Moller vertex KF", "TC Moller vertex KF", "Other electron KF", "UC VC vertex KF", "Final state particle GBL", "UC V0 vertex GBL",
"BSC V0 vertex GBL", "TC V0 vertex GBL", "UC Moller vertex", "BSC Moller vertex", "TC Moller vertex", "Other electron GBL",
"UC VC vertex GBL"]
All_Names = list(dfx.GetColumnNames())
Store_Names = []
ii=0
remove_names = [] # ["v0_","hodo_","ext_trigger","part_","rf_","svt"]
for n in All_Names:
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):
        Store_Names.append(s)
        dat_type = dfx.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_nhits             [RVec<int>           ] ecal_cluster_seed_energy       [RVec<double>        ] 
ecal_cluster_seed_index        [RVec<int>           ] ecal_cluster_seed_ix           [RVec<int>           ] 
ecal_cluster_seed_iy           [RVec<int>           ] ecal_cluster_time              [RVec<double>        ] 
ecal_cluster_uncor_energy      [RVec<double>        ] ecal_cluster_uncor_hits        [RVec<vector<int>>   ] 
ecal_cluster_uncor_nhits       [RVec<int>           ] ecal_cluster_uncor_seed_energy [RVec<double>        ] 
ecal_cluster_uncor_seed_index  [RVec<int>           ] ecal_cluster_uncor_seed_ix     [RVec<int>           ] 
ecal_cluster_uncor_seed_iy     [RVec<int>           ] ecal_cluster_uncor_time        [RVec<double>        ] 
ecal_cluster_uncor_x           [RVec<double>        ] ecal_cluster_uncor_y           [RVec<double>        ] 
ecal_cluster_uncor_

In [7]:
num = dfx.Count()
num2 = dfxx.Count()
num3 = dfm.Count()
print(f" Num x = {num.GetValue()}   Num2 xx = {num2.GetValue()}  Num3 m = {num3.GetValue()}")

 Num x = 397975   Num2 xx = 2180016  Num3 m = 3999682


In [8]:
 # Example - Check that the charge of electrons is -1, first with indirection, then with a direct selection on the part_pdg.
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<e_part_idx.size();++i){out.push_back(part_charge[e_part_idx[i]]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
# h_tmp = dfx.Define("e_charge","vector<int> out;for(int i=0;i<part_charge.size();++i){if(part_pdg[i]==11) out.push_back(part_charge[i]);} return out;").Histo1D(("h_tmp","e_charge",5,-2.5,2.5),"e_charge")
h_n_part = dfx.Define("n_part","return part_pdg.size()").Histo1D(("h_n_part","Number of particles",11,-0.5,10.5),"n_part")
h_n_neg_track = dfx.Define("n_neg_track","int count=0;for(int i=0;i<track_omega.size();++i){if(track_omega[i]>0 && track_type[i]==1)count++;} return count;").Histo1D(("h_n_neg_track","Number of - tracks",11,-0.5,10.5),"n_neg_track")
h_n_electron = dfx.Define("n_electron","int out=0; for(int i=0; i<part_pdg.size(); ++i){if(part_pdg[i]==11) out++;} return out;").Histo1D(("h_n_electron","Number of e-",11,-0.5,10.5),"n_electron")
h_n_moller = dfx.Define("n_moller","return electron_pairs.size()").Histo1D(("h_n_moller","Number of Moller pairs",11,-0.5,10.5),"n_moller")

h_track_time_diff = dfx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("h_track_time_diff","track_time_diff",500,-50.,50.),"track_time_diff")
hx_track_time_diff = dfxx.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hx_track_time_diff","track_time_diff",500,-50.,50.),"track_time_diff")
hm_track_time_diff = dfm.Define("track_time_diff","vector<double> out; for(auto p: electron_pairs){out.push_back(track_time[part_track[p.first]] - track_time[part_track[p.second]]);} return out;").Histo1D(("hm_track_time_diff","track_time_diff",500,-50.,50.),"track_time_diff")


h_track_chi2 = dfx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("h_track_chi2","track_chi2",500,0.,10.),"t_chi2")
hx_track_chi2 = dfxx.Define("t_chi2","vector<double> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_chi2[t]);} return out;").Histo1D(("xh_track_chi2","track_chi2",500,0.,10.),"t_chi2")

h_track_nhit = dfx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("h_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")
hx_track_nhit = dfxx.Define("t_nhit","vector<int> out; std::set<int> tracks; for(auto p: electron_pairs){tracks.insert(part_track[p.first]); tracks.insert(part_track[p.second]);};for(auto t: tracks){out.push_back(track_n_hits[t]);} return out;").Histo1D(("hx_track_nhit","track_nhit",15,-0.5,14.5),"t_nhit")

h_mom_sum = dfx.Define("mom_sum","vector<double> out; for(int i=0; i<p4v1.size(); ++i){out.push_back((p4v1[i].Vect()+p4v2[i].Vect()).Mag());} return out;").Histo1D(("h_mom_sum","mom sum",500,0.,6.0),"mom_sum")
h_mom_sum_r1 = dfx.Define("mom_sum","vector<double> out; for(int i=0; i<r1_p4v1.size(); ++i){out.push_back((r1_p4v1[i].Vect()+r1_p4v2[i].Vect()).Mag());} return out;").Histo1D(("h_mom_sum_r1","mom sum r1",500,0.,6.0),"mom_sum")
h_mom_sum_r2 = dfx.Define("mom_sum","vector<double> out; for(int i=0; i<r2_p4v1.size(); ++i){out.push_back((r2_p4v1[i].Vect()+r2_p4v2[i].Vect()).Mag());} return out;").Histo1D(("h_mom_sum_r2","mom sum r2",500,0.,6.0),"mom_sum")
h_mom_sum_r3 = dfx.Define("mom_sum","vector<double> out; for(int i=0; i<r3_p4v1.size(); ++i){out.push_back((r3_p4v1[i].Vect()+r3_p4v2[i].Vect()).Mag());} return out;").Histo1D(("h_mom_sum_r3","mom sum r3",500,0.,6.0),"mom_sum")
h_mom_sum_r4 = dfx.Define("mom_sum","vector<double> out; for(int i=0; i<r4_p4v1.size(); ++i){out.push_back((r4_p4v1[i].Vect()+r4_p4v2[i].Vect()).Mag());} return out;").Histo1D(("h_mom_sum_r4","mom sum r4",500,0.,6.0),"mom_sum")


hx_mom_sum = dfxx.Define("mom_sum","vector<double> out; for(int i=0; i<p4v1.size(); ++i){out.push_back((p4v1[i].Vect()+p4v2[i].Vect()).Mag());} return out;").Histo1D(("hx_mom_sum","mom sum",500,0.,6.0),"mom_sum")
hx_mom_sum_r1 = dfxx.Define("mom_sum","vector<double> out; for(int i=0; i<r1_p4v1.size(); ++i){out.push_back((r1_p4v1[i].Vect()+r1_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hx_mom_sum_r1","mom sum r1",500,0.,6.0),"mom_sum")
hx_mom_sum_r2 = dfxx.Define("mom_sum","vector<double> out; for(int i=0; i<r2_p4v1.size(); ++i){out.push_back((r2_p4v1[i].Vect()+r2_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hx_mom_sum_r2","mom sum r2",500,0.,6.0),"mom_sum")
hx_mom_sum_r3 = dfxx.Define("mom_sum","vector<double> out; for(int i=0; i<r3_p4v1.size(); ++i){out.push_back((r3_p4v1[i].Vect()+r3_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hx_mom_sum_r3","mom sum r3",500,0.,6.0),"mom_sum")
hx_mom_sum_r4 = dfxx.Define("mom_sum","vector<double> out; for(int i=0; i<r4_p4v1.size(); ++i){out.push_back((r4_p4v1[i].Vect()+r4_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hx_mom_sum_r4","mom sum r4",500,0.,6.0),"mom_sum")


hm_mom_sum = dfm.Define("mom_sum","vector<double> out; for(int i=0; i<p4v1.size(); ++i){out.push_back((p4v1[i].Vect()+p4v2[i].Vect()).Mag());} return out;").Histo1D(("hm_mom_sum","mom sum",500,0.,6.0),"mom_sum")
hm_mom_sum_r1 = dfm.Define("mom_sum","vector<double> out; for(int i=0; i<r1_p4v1.size(); ++i){out.push_back((r1_p4v1[i].Vect()+r1_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hm_mom_sum_r1","mom sum r1",500,0.,6.0),"mom_sum")
hm_mom_sum_r2 = dfm.Define("mom_sum","vector<double> out; for(int i=0; i<r2_p4v1.size(); ++i){out.push_back((r2_p4v1[i].Vect()+r2_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hm_mom_sum_r2","mom sum r2",500,0.,6.0),"mom_sum")
hm_mom_sum_r3 = dfm.Define("mom_sum","vector<double> out; for(int i=0; i<r3_p4v1.size(); ++i){out.push_back((r3_p4v1[i].Vect()+r3_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hm_mom_sum_r3","mom sum r3",500,0.,6.0),"mom_sum")
hm_mom_sum_r4 = dfm.Define("mom_sum","vector<double> out; for(int i=0; i<r4_p4v1.size(); ++i){out.push_back((r4_p4v1[i].Vect()+r4_p4v2[i].Vect()).Mag());} return out;").Histo1D(("hm_mom_sum_r4","mom sum r4",500,0.,6.0),"mom_sum")


h_beam_theta_x = dfx.Histo1D(("h_beam_theta_x","Beam Angle #theta_{x}",1000,-0.1,0.1),"p4beam_theta_x")
h_beam_theta_x_r2 = dfx.Filter("el_pairs_r2.size()>=1").Histo1D(("h_beam_theta_x_r2","Beam Angle #theta_{x}",1000,-0.1,0.1),"r2_p4beam_theta_x")
h_beam_theta_x_r3 = dfx.Filter("el_pairs_r3.size()>=1").Histo1D(("h_beam_theta_x_r3","Beam Angle #theta_{x}",1000,-0.1,0.1),"r3_p4beam_theta_x")
hx_beam_theta_x = dfxx.Histo1D(("hx_beam_theta_x","Beam Angle #theta_{x}",1000,-0.1,0.1),"p4beam_theta_x")

h_ee_thtE1 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht1","p4E1")
h_ee_thtE2 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"p4tht2","p4E2")
h_ee_thth = dfx.Histo2D(("h_ee_thth","EE theta-theta;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"p4tht1","p4tht2")
#h_ee_thtE1_r1 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1 R1;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r1_p4tht1","r1_p4E1")
#h_ee_thtE2_r1 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2 R1;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r1_p4tht2","r1_p4E2")
#h_ee_thth_r1 = dfx.Histo2D(("h_ee_thth","EE theta-theta R1;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r1_p4tht1","r1_p4tht2")
h_ee_thtE1_r2 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1 R2;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r2_p4tht1","r2_p4E1")
h_ee_thtE2_r2 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2 R2;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r2_p4tht2","r2_p4E2")
h_ee_thth_r2 = dfx.Histo2D(("h_ee_thth","EE theta-theta R2;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r2_p4tht1","r2_p4tht2")
h_ee_thtE1_r3 = dfx.Histo2D(("h_ee_thtE1","EE theta-E 1 R2+R3;#theta_{1} [rad];E_{1} [GeV]",500,0.,0.1,500,0.,2.5),"r3_p4tht1","r3_p4E1")
h_ee_thtE2_r3 = dfx.Histo2D(("h_ee_thtE2","EE theta-E 2 R2+R3;#theta_{2} [rad];E_{2} [GeV]",500,0.,0.1,500,0.,2.5),"r3_p4tht2","r3_p4E2")
h_ee_thth_r3 = dfx.Histo2D(("h_ee_thth","EE theta-theta R2+R3;#theta_{1} [rad];#theta_{2} [rad]",500,0.,0.1,500,0.,0.1),"r3_p4tht1","r3_p4tht2")

In [9]:
%jsroot on
cc0 = R.TCanvas("cc0","cc0",1200,800)
cc0.Divide(3,2)
cc0.cd(1)
h_n_moller.SetTitle("Particle counts in Moller events")
h_n_moller.SetStats(0)
h_n_moller.SetLineWidth(2)
h_n_moller.SetLineColor(R.kCyan)
h_n_moller.Draw("same")
h_n_electron.SetLineWidth(3)
h_n_electron.SetLineColor(R.kBlue)
h_n_electron.Draw("same")
h_n_neg_track.SetLineColor(R.kRed)
h_n_neg_track.Draw("same")
h_n_part.SetLineWidth(1)
h_n_part.SetLineColor(R.kGreen+2)
h_n_part.Draw("same")
legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_n_part.GetPtr(),"any particle","l")
legend.AddEntry(h_n_neg_track.GetPtr(),"neg. tracks","l")
legend.AddEntry(h_n_electron.GetPtr(),"electron","l")
legend.AddEntry(h_n_moller.GetPtr(),"Moller","l")
legend.Draw()
cc0.cd(2)
h_track_time_diff.SetLineWidth(1)
h_track_time_diff.SetLineColor(R.kGreen+2)
h_track_time_diff.Draw()
hx_track_time_diff.SetLineColor(R.kRed)
hx_track_time_diff.Draw("same")
# hm_track_time_diff.SetLineColor(R.kAzure)
# hm_track_time_diff.Draw("same")
pad6 = cc0.cd(3)
pad6.SetLogy()
hx_beam_theta_x.SetStats(0)
hx_beam_theta_x.SetLineColor(R.kRed)
hx_beam_theta_x.Draw("same")
#hx_beam_theta_x.Fit("gaus")

h_beam_theta_x.SetStats(0)
h_beam_theta_x.SetLineWidth(1)
h_beam_theta_x.SetLineColor(R.kGreen+2)
h_beam_theta_x.Draw("same")
#h_beam_theta_x.Fit("gaus")

h_beam_theta_x_r2.SetStats(0)
h_beam_theta_x_r2.SetLineWidth(1)
h_beam_theta_x_r2.SetLineColor(R.kBlue)
h_beam_theta_x_r2.Draw("same")
#h_beam_theta_x_r2.Fit("gaus")

h_beam_theta_x_r3.SetStats(0)
h_beam_theta_x_r3.SetLineWidth(1)
h_beam_theta_x_r3.SetLineColor(R.kOrange)
h_beam_theta_x_r3.Draw("same")
#h_beam_theta_x_r3.Fit("gaus")

leg2 = R.TLegend(0.7,0.7,0.9,0.9)
leg2.AddEntry(hx_beam_theta_x.GetPtr(),"Not Moller","l")
leg2.AddEntry(h_beam_theta_x.GetPtr(),"Moller","l")
leg2.AddEntry(h_beam_theta_x_r2.GetPtr(),"Moller r2","l")
leg2.AddEntry(h_beam_theta_x_r3.GetPtr(),"Moller r2+r3","l")
leg2.Draw()

pad2 = cc0.cd(4)
pad2.SetLogz()
h_ee_thth_r3.SetStats(0)
h_ee_thth_r3.Draw("colz")
l1 = R.TLine(0.,0.04,0.04,0.)
# l1.SetLineColor(R.kRed)
# l1.SetLineWidth(2)
# l1.Draw()
# l2 = R.TLine(0.,0.048,0.048,0.)
# l2.SetLineColor(R.kRed)
# l2.SetLineWidth(2)
# l2.Draw()
f1 = R.TF1("f1","2*asin(0.00011108696/sin(x/2))",0.0044,0.1)
f1.SetLineColor(R.kGreen)
f1.Draw("same")

pad3 = cc0.cd(5)
pad3.SetLogz()
h_ee_thtE1_r3.SetStats(0)
h_ee_thtE1_r3.Draw("colz")
f2 = R.TF1("f2","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f2.SetLineColor(R.kRed)
f2.Draw("same")

pad4 = cc0.cd(6)
#f1 = R.TF1("f1","2*asin(0.11108696/sin(x/2))",0.0001,0.1)
pad4.SetLogz()
h_ee_thtE2_r3.SetStats(0)
h_ee_thtE2_r3.Draw("colz")
f3 = R.TF1("f3","2.3/(1+ (2.3/0.000511)*(1-cos(x)))",0.005,0.1)
f3.SetLineColor(R.kRed)
f3.Draw("same")


cc0.Draw()

In [10]:
cc1 = R.TCanvas("cc1","cc1",1200,400)
cc1.Divide(3,1)

cc1.cd(1)
h_mom_sum.SetStats(0)
h_mom_sum.SetTitle("Momentum sum, Moller pairs")
h_mom_sum.SetLineWidth(1)
h_mom_sum.SetLineColor(R.kBlue)
h_mom_sum.Draw()
h_mom_sum_r1.SetStats(0)
h_mom_sum_r1.SetLineColor(R.kGreen+2)
h_mom_sum_r1.Draw("same")
h_mom_sum_r2.SetStats(0)
h_mom_sum_r2.SetLineColor(R.kRed)
h_mom_sum_r2.Draw("same")
h_mom_sum_r3.SetStats(0)
h_mom_sum_r3.SetLineColor(R.kOrange)
h_mom_sum_r3.Draw("same")
h_mom_sum_r4.SetStats(0)
h_mom_sum_r4.SetLineColor(R.kCyan)
h_mom_sum_r4.Draw("same")
legend2 = R.TLegend(0.1,0.7,0.4,0.9)
legend2.AddEntry(h_mom_sum.GetPtr(),"Moller","l")
legend2.AddEntry(h_mom_sum_r1.GetPtr(),"Moller r1","l")
legend2.AddEntry(h_mom_sum_r2.GetPtr(),"Moller r1+r2","l")
legend2.AddEntry(h_mom_sum_r3.GetPtr(),"Moller r1+r2+r3","l")
legend2.AddEntry(h_mom_sum_r4.GetPtr(),"Moller ...r3+r4","l")
legend2.Draw()

cc1.cd(2)
hx_mom_sum.SetStats(0)
hx_mom_sum.SetTitle("Momentum sum, Not Moller pairs")
hx_mom_sum.SetLineWidth(1)
hx_mom_sum.SetLineColor(R.kBlue)
hx_mom_sum.Draw()
hx_mom_sum_r1.SetStats(0)
hx_mom_sum_r1.SetLineColor(R.kGreen+2)
hx_mom_sum_r1.Draw("same")
hx_mom_sum_r2.SetStats(0)
hx_mom_sum_r2.SetLineColor(R.kRed)
hx_mom_sum_r2.Draw("same")
hx_mom_sum_r3.SetStats(0)
hx_mom_sum_r3.SetLineColor(R.kOrange)
hx_mom_sum_r3.Draw("same")
hx_mom_sum_r3.Draw("same")
hx_mom_sum_r4.SetStats(0)
hx_mom_sum_r4.SetLineColor(R.kCyan)
hx_mom_sum_r4.Draw("same")

legend3 = R.TLegend(0.1,0.7,0.4,0.9)
legend3.AddEntry(hx_mom_sum.GetPtr(),"Not Moller","l")
legend3.AddEntry(hx_mom_sum_r1.GetPtr(),"Not Moller r1","l")
legend3.AddEntry(hx_mom_sum_r2.GetPtr(),"Not Moller r2","l")
legend3.AddEntry(hx_mom_sum_r3.GetPtr(),"Not Moller r2+r3","l")
legend3.AddEntry(hx_mom_sum_r4.GetPtr(),"Moller ...r3+r4","l")
legend3.Draw()

cc1.cd(3)

hm_mom_sum.SetStats(0)
hm_mom_sum.SetTitle("Momentum sum, Mixed pairs")
hm_mom_sum.SetLineWidth(1)
hm_mom_sum.SetLineColor(R.kBlue)
hm_mom_sum.Draw()
hm_mom_sum_r1.SetStats(0)
hm_mom_sum_r1.SetLineColor(R.kGreen+2)
hm_mom_sum_r1.Draw("same")
hm_mom_sum_r2.SetStats(0)
hm_mom_sum_r2.SetLineColor(R.kRed)
hm_mom_sum_r2.Draw("same")
hm_mom_sum_r3.SetStats(0)
hm_mom_sum_r3.SetLineColor(R.kOrange)
hm_mom_sum_r3.Draw("same")
hm_mom_sum_r4.SetStats(0)
hm_mom_sum_r4.SetLineColor(R.kCyan)
hm_mom_sum_r4.Draw("same")

legend4 = R.TLegend(0.1,0.7,0.4,0.9)
legend4.AddEntry(hm_mom_sum.GetPtr(),"Mixed","l")
legend4.AddEntry(hm_mom_sum_r1.GetPtr(),"Mixed r1","l")
legend4.AddEntry(hm_mom_sum_r2.GetPtr(),"Mixed r2","l")
legend4.AddEntry(hm_mom_sum_r3.GetPtr(),"Mixed r2+r3","l")
legend4.AddEntry(hm_mom_sum_r4.GetPtr(),"Moller ...r3+r4","l")
legend4.Draw()

cc1.Draw()

In [11]:
h_ee_mass_r1 = dfx.Histo1D(("h_ee_mass_r1","EE inv mass R1; M_{inv} [GeV]",500,0.,0.2),"r1_p4minv")
hx_ee_mass_r1 = dfxx.Histo1D(("hx_ee_mass_r1","EE inv mass R1; M_{inv} [GeV]",500,0.,0.2),"r1_p4minv")
hm_ee_mass_r1 = dfm.Histo1D(("hm_ee_mass_r1","EE inv mass R1; M_{inv} [GeV]",500,0.,0.2),"r1_p4minv")

h_ee_mass_r2 = dfx.Histo1D(("h_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")
hx_ee_mass_r2 = dfxx.Histo1D(("hx_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")
hm_ee_mass_r2 = dfm.Histo1D(("hm_ee_mass_r2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r2_p4minv")

h_ee_mass_r3 = dfx.Histo1D(("h_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")
hx_ee_mass_r3 = dfxx.Histo1D(("hx_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")
hm_ee_mass_r3 = dfm.Histo1D(("hm_ee_mass_r3","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r3_p4minv")

h_ee_mass_r4 = dfx.Histo1D(("h_ee_mass_r4","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r4_p4minv")
hx_ee_mass_r4 = dfxx.Histo1D(("hx_ee_mass_r4","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r4_p4minv")
hm_ee_mass_r4 = dfm.Histo1D(("hm_ee_mass_r4","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"r4_p4minv")

h_ee_mass_rx1 = dfx.Histo1D(("h_ee_mass_rx1","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx1_p4minv")
hx_ee_mass_rx1 = dfxx.Histo1D(("hx_ee_mass_rx1","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx1_p4minv")
hm_ee_mass_rx1 = dfm.Histo1D(("hm_ee_mass_rx1","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx1_p4minv")

h_ee_mass_rx2 = dfx.Histo1D(("h_ee_mass_rx2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx2_p4minv")
hx_ee_mass_rx2 = dfxx.Histo1D(("hx_ee_mass_rx2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx2_p4minv")
hm_ee_mass_rx2 = dfm.Histo1D(("hm_ee_mass_rx2","EE inv mass; M_{inv} [GeV]",500,0.,0.2),"rx2_p4minv")



In [12]:
cc2 = R.TCanvas("cc2","cc2",1200,400)
cc2.Divide(3,1)
pad1= cc2.cd(1)
# pad1.SetLogy()
h_ee_mass_r1.SetTitle("M_{inv} Moller")
h_ee_mass_r1.SetStats(0)
h_ee_mass_r1.SetLineColor(R.kGreen+2)
h_ee_mass_r1.SetLineWidth(2)
h_ee_mass_r1.Draw()

h_ee_mass_r2.SetStats(0)
h_ee_mass_r2.SetLineColor(R.kRed)
h_ee_mass_r2.SetLineWidth(2)
h_ee_mass_r2.Draw("same")

h_ee_mass_r3.SetStats(0)
h_ee_mass_r3.SetLineColor(R.kOrange)
h_ee_mass_r3.SetLineWidth(2)
h_ee_mass_r3.Draw("same")

h_ee_mass_r4.SetStats(0)
h_ee_mass_r4.SetLineColor(R.kCyan)
h_ee_mass_r4.SetLineWidth(2)
h_ee_mass_r4.Draw("same")

legend = R.TLegend(0.6,0.7,0.9,0.9)
legend.AddEntry(h_ee_mass_r1.GetPtr(),"Moller r1","l")
legend.AddEntry(h_ee_mass_r2.GetPtr(),"Moller r1+r2","l")
legend.AddEntry(h_ee_mass_r3.GetPtr(),"Moller ...r2+r3","l")
legend.AddEntry(h_ee_mass_r4.GetPtr(),"Moller ...r3+r4","l")
legend.Draw()

pad2 = cc2.cd(2)

hx_ee_mass_r1.SetTitle("M_{inv} Not Moller")
hx_ee_mass_r1.SetStats(0)
hx_ee_mass_r1.SetLineColor(R.kGreen+2)
hx_ee_mass_r1.SetLineWidth(2)
hx_ee_mass_r1.Draw()

hx_ee_mass_r2.SetStats(0)
hx_ee_mass_r2.Draw("same")

hx_ee_mass_r2.SetStats(0)
hx_ee_mass_r2.SetLineColor(R.kRed)
hx_ee_mass_r2.SetLineWidth(2)
hx_ee_mass_r2.Draw("same")

hx_ee_mass_r3.SetStats(0)
hx_ee_mass_r3.SetLineColor(R.kOrange)
hx_ee_mass_r3.SetLineWidth(2)
hx_ee_mass_r3.Draw("same")

hx_ee_mass_r4.SetStats(0)
hx_ee_mass_r4.SetLineColor(R.kCyan)
hx_ee_mass_r4.SetLineWidth(2)
hx_ee_mass_r4.Draw("same")

legend2 = R.TLegend(0.6,0.7,0.9,0.9)
legend2.AddEntry(hx_ee_mass_r1.GetPtr(),"Not Moller r1","l")
legend2.AddEntry(hx_ee_mass_r2.GetPtr(),"Not Moller r1+r2","l")
legend2.AddEntry(hx_ee_mass_r3.GetPtr(),"Not Moller ...r2+r3","l")
legend2.AddEntry(hx_ee_mass_r4.GetPtr(),"Not Moller ...r3+r4","l")

legend2.Draw()

pad3 = cc2.cd(3)

hm_ee_mass_r1.SetTitle("M_{inv} Mixed")
hm_ee_mass_r1.SetStats(0)
hm_ee_mass_r1.SetLineColor(R.kGreen+2)
hm_ee_mass_r1.SetLineWidth(2)
hm_ee_mass_r1.Draw()

hm_ee_mass_r2.SetStats(0)
hm_ee_mass_r2.Draw("same")

hm_ee_mass_r2.SetStats(0)
hm_ee_mass_r2.SetLineColor(R.kRed)
hm_ee_mass_r2.SetLineWidth(2)
hm_ee_mass_r2.Draw("same")

hm_ee_mass_r3.SetStats(0)
hm_ee_mass_r3.SetLineColor(R.kOrange)
hm_ee_mass_r3.SetLineWidth(2)
hm_ee_mass_r3.Draw("same")

hm_ee_mass_r4.SetStats(0)
hm_ee_mass_r4.SetLineColor(R.kCyan)
hm_ee_mass_r4.SetLineWidth(2)
hm_ee_mass_r4.Draw("same")

legend3 = R.TLegend(0.6,0.7,0.9,0.9)
legend3.AddEntry(hx_ee_mass_r1.GetPtr(),"Mixed r1","l")
legend3.AddEntry(hx_ee_mass_r2.GetPtr(),"Mixed r1+r2","l")
legend3.AddEntry(hx_ee_mass_r3.GetPtr(),"Mixed ...r2+r3","l")
legend3.AddEntry(hx_ee_mass_r4.GetPtr(),"Mixed ...r3+r4","l")

legend3.Draw()


cc2.Draw()

In [14]:
cc3 = R.TCanvas("cc3","cc3",1200,500)
cc3.Divide(2,1)

pad1 = cc3.cd(1)

hx_ee_mass_r3.SetTitle("M_{inv} Moller r1+r2+r3")
hx_ee_mass_r3.SetLineColor(R.kRed)
hx_ee_mass_r3.SetLineWidth(2)
hx_ee_mass_r3.Draw("same")


hm_ee_mass_r3.SetLineColor(R.kOrange)
hm_ee_mass_r3.SetLineWidth(2)
hm_ee_mass_r3.Draw("same")

h_ee_mass_r3.SetTitle("M_{inv} Moller r1+r2+r3")
h_ee_mass_r3.SetLineColor(R.kBlue)
h_ee_mass_r3.SetLineWidth(2)
h_ee_mass_r3.Draw("same")


legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_ee_mass_r3.GetPtr(),"Moller","l")
legend.AddEntry(hx_ee_mass_r3.GetPtr(),"Not Moller","l")
legend.AddEntry(hm_ee_mass_r3.GetPtr(),"Mixed","l")

legend.Draw()

pad2 = cc3.cd(2)

h_ee_mass_r4.SetTitle("M_{inv} Moller r1+..+r4")
h_ee_mass_r4.SetLineColor(R.kBlue)
h_ee_mass_r4.SetLineWidth(2)
h_ee_mass_r4.Draw("same")

hx_ee_mass_r4.SetLineColor(R.kRed)
hx_ee_mass_r4.SetLineWidth(2)
hx_ee_mass_r4.Draw("same")

hm_ee_mass_r4.SetLineColor(R.kOrange)
hm_ee_mass_r4.SetLineWidth(2)
hm_ee_mass_r4.Draw("same")

cc3.Draw()



Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc3


In [16]:
cc4 = R.TCanvas("cc4","cc4",1200,500)
cc4.Divide(2,1)

pad1 = cc4.cd(1)

h_ee_mass_rx1.SetTitle("M_{inv} Moller rx1")
h_ee_mass_rx1.SetStats(0)
h_ee_mass_rx1.SetLineColor(R.kBlue)
h_ee_mass_rx1.SetLineWidth(2)
h_ee_mass_rx1.Draw("same")

hx_ee_mass_rx1.SetLineColor(R.kRed)
hx_ee_mass_rx1.SetLineWidth(2)
hx_ee_mass_rx1.Draw("same")

hm_ee_mass_rx1.SetLineColor(R.kOrange)
hm_ee_mass_rx1.SetLineWidth(2)
hm_ee_mass_rx1.Draw("same")

legend = R.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(h_ee_mass_rx1.GetPtr(),"Moller","l")
legend.AddEntry(hx_ee_mass_rx1.GetPtr(),"Not Moller","l")
legend.AddEntry(hm_ee_mass_rx1.GetPtr(),"Mixed","l")
legend.Draw()

pad2 = cc4.cd(2)

h_ee_mass_rx2.SetTitle("M_{inv} Moller rx2")
h_ee_mass_rx2.SetStats(0)
h_ee_mass_rx2.SetLineColor(R.kBlue)
h_ee_mass_rx2.SetLineWidth(2)
h_ee_mass_rx2.Draw("same")

hx_ee_mass_rx2.SetLineColor(R.kRed)
hx_ee_mass_rx2.SetLineWidth(2)
hx_ee_mass_rx2.Draw("same")

hm_ee_mass_rx2.SetLineColor(R.kOrange)
hm_ee_mass_rx2.SetLineWidth(2)
hm_ee_mass_rx2.Draw("same")

cc4.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc4
